# 02 Azure OpenAI .NET SDK

## Intro 

The Azure OpenAI SDK for .NET is a software development kit that allows .NET developers to integrate OpenAI's language models into their applications. It is provided as a NuGet package and at the time of recording of the training available in the version 1.0.0-beta.7.

The SDK provides the object `OpenAIClient` which can be instantiated for making API calls to access the capabilities of OpenAI's language models, such as text generation. 

## Step 1 - Read Environment / Create OpenAIClient instance

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.7"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;

string _configurationFile = @"../01/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY");
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT");
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created...");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.7 DotNetEnv, 2.5.0

OpenAI Client created...


## Step 2 - Define system message & Summarization text

In [2]:
string systemMessage = @"
    Summarize the provided unstructured text in 3 easy to understand sentences. 
    The sentences need to be short and provide the most important content of the provided text.";

string textToSummarize = @"
    The Super Bowl is the annual American football game that determines the champion of the National Football League (NFL). 
    The game culminates a season that begins in the previous calendar year, and is the conclusion of the NFL playoffs. 
    The winner receives the Vince Lombardi Trophy. 
    The contest is held in an American city, chosen three to four years beforehand, usually at warm-weather sites or domed stadiums.
    Since January 1971, the winner of the American Football Conference (AFC) Championship Game has faced the winner of the 
    National Football Conference  (NFC) Championship Game in the culmination of the NFL playoffs. 
    Before the 1970 merger between the American Football League (AFL) and the National Football League (NFL), 
    the two leagues met in four such contests. 
    The first two were marketed as the AFL NFL World Championship Game, but were also casually referred to as the 
    Super Bowl game during the television  broadcast. 
    Super Bowl III in January 1969 was the first such game that carried the Super Bowl moniker in official marketing; 
    the names Super Bowl and Super Bowl II were retroactively applied to the first two games. 
    A total of 20 franchises, including teams that have relocated to another city or changed their name, have won the Super Bowl.
    There are four NFL teams that have never  appeared in a Super Bowl: the Cleveland Browns, Detroit Lions, Jacksonville Jaguars, 
    and Houston Texans, though both the Browns (1950, 1954, 1955, 1964) and Lions (1935, 1952, 1953, 1957) had won NFL Championship 
    Games prior to the  creation of the Super Bowl in the 1966 season. 
    The 1972 Dolphins capped off the only perfect season in NFL history with their  victory in Super Bowl VII. 
    Only two franchises have ever won the Super Bowl while hosting at their home stadiums: the Tampa Bay Buccaneers in Super Bowl 
    LV and the Los Angeles Rams in Super Bowl LVI.";

Console.WriteLine($"System message & text to summarize defined ...");


System message & text to summarize defined ...


## Step 3 - Call LLM endpoint

In [3]:

//Compose Chat
ChatCompletionsOptions chatCompletionsOptions = new ChatCompletionsOptions();
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.System, systemMessage));
chatCompletionsOptions.Messages.Add(new ChatMessage(ChatRole.User, textToSummarize));

//Request Properties
chatCompletionsOptions.MaxTokens = 500;
chatCompletionsOptions.Temperature = 0.0f;
chatCompletionsOptions.NucleusSamplingFactor = 0.0f;
chatCompletionsOptions.FrequencyPenalty = 0.7f;
chatCompletionsOptions.PresencePenalty = 0.7f;

// Call the Model ChatCompletions endpoint
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(
    chatCompletionDeploymentName, 
    chatCompletionsOptions);

ChatChoice chatChoice = response.Value.Choices[0];
Console.WriteLine("Summarization:");
foreach (string line in chatChoice.Message.Content.Split("."))
{
    Console.WriteLine(line.Trim());
}


Summarization:
The Super Bowl is the annual American football game that determines the NFL champion, culminating a season that begins in the previous year
The winner receives the Vince Lombardi Trophy and it's held in an American city chosen years beforehand
There are four NFL teams that have never appeared in a Super Bowl: Cleveland Browns, Detroit Lions, Jacksonville Jaguars, and Houston Texans



## Step 4 (optional) - Response streaming

In [4]:
Response<StreamingChatCompletions> streamingResponse = await openAIClient.GetChatCompletionsStreamingAsync(chatCompletionDeploymentName, chatCompletionsOptions);

string responseChunk = string.Empty;

using (streamingResponse.Value) {
    await foreach(StreamingChatChoice chatChoice in streamingResponse.Value.GetChoicesStreaming()){
        await foreach(ChatMessage chatMessage in chatChoice.GetMessageStreaming()){
            Console.WriteLine($"Arrived chunk: {chatMessage.Content}");
            responseChunk += chatMessage.Content;
        }
    }
}


Arrived chunk: 
Arrived chunk: The
Arrived chunk:  Super
Arrived chunk:  Bowl
Arrived chunk:  is
Arrived chunk:  the
Arrived chunk:  annual
Arrived chunk:  American
Arrived chunk:  football
Arrived chunk:  game
Arrived chunk:  that
Arrived chunk:  determines
Arrived chunk:  the
Arrived chunk:  NFL
Arrived chunk:  champion
Arrived chunk: ,
Arrived chunk:  cul
Arrived chunk: minating
Arrived chunk:  a
Arrived chunk:  season
Arrived chunk:  that
Arrived chunk:  begins
Arrived chunk:  in
Arrived chunk:  the
Arrived chunk:  previous
Arrived chunk:  year
Arrived chunk: .
Arrived chunk:  The
Arrived chunk:  winner
Arrived chunk:  receives
Arrived chunk:  the
Arrived chunk:  Vince
Arrived chunk:  Lomb
Arrived chunk: ardi
Arrived chunk:  Trophy
Arrived chunk:  and
Arrived chunk:  it
Arrived chunk: 's
Arrived chunk:  held
Arrived chunk:  in
Arrived chunk:  an
Arrived chunk:  American
Arrived chunk:  city
Arrived chunk:  chosen
Arrived chunk:  years
Arrived chunk:  beforehand
Arrived chunk: .
Arr